# Process data

## Imports and setup

In [1]:
import glob
from multiprocessing import Process
import os

from sessionlists import (dist60, dist90, dist120, TM20, TM10, TM2, TMrev2, TMrev10, TMrev20, buggedSessions)
from process_functions import *

In [2]:
# Define folder with data
root = "/home/thomas/Desktop/_ALLDATA"

print("Path to data is: %s"%root)
print(f'Found {len(glob.glob(root+"/*"))} rats in the data folder')
animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]

Path to data is: /home/thomas/Desktop/_ALLDATA
Found 34 rats in the data folder


# Process all sessions

In [3]:
# Define the sessions to process. Can also filter by animal, or session name
# Maybe better not to process everything at once, so try chuncking

# all sessions
SESSIONS_TO_PROCESS = dist60 + dist90 + dist120 + TM20 + TM10 + TM2 + TMrev2 + TMrev10 + TMrev20

# e.g. only process dist60 sessions
# SESSIONS_TO_PROCESS = dist60

# e.g. only process RatM00
# SESSIONS_TO_PROCESS = matchsession("RatM00", dist60 + dist90 + dist120 + TM20 + TM10 + TM2 + TMrev2 + TMrev10 + TMrev20)

# e.g. only process RatM00 in the dist60 condition
# SESSIONS_TO_PROCESS = matchsession("RatM00", dist60)

# only one session of RatM00
animalList = ["RatM00"]
SESSIONS_TO_PROCESS = ["RatM00_2021_07_22_16_13_03"]

In [ ]:
# Multiprocessing to speed up the processing

p = {}
for index, animal in enumerate(animalList):
    p[animal] = Process(target = processData, args=(root,  # path to data
                                                    animal,  # animal name
                                                    matchsession(animal, SESSIONS_TO_PROCESS),  # list of sessions to process for this animal
                                                    index,  # for colors, etc.
                                                    buggedSessions,  # list of sessions to skip
                                                    True,  # redo the complete processing of the data
                                                    True,  # redraw the recap figure (big so takes time)
                                                    False,  # show the figures while processing (adds time)
                                                    True,  # recompute the run/inter-run mask (takes time)
                                                    ))
for index, animal in enumerate(animalList):
    p[animal].start()
for index, animal in enumerate(animalList):
    p[animal].join()

In [ ]:
# Could also use this to process only one session, and loop through the animals and sessions
animal = "RatM00"
processData(root, animal, matchsession(animal, SESSIONS_TO_PROCESS), 1,
            buggedSessions, True, True, True, True)

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

# Load processed data

In [6]:
params = {}
smoothMask = {}
nb_runsBin = {}
rat_markers = {}
speedRunToLeftBin, speedRunToRightBin = {}, {}
timeRunToLeftBin, timeRunToRightBin = {}, {}
maxSpeedLeftBin, maxSpeedRightBin = {}, {}
timeStayInLeftBin, timeStayInRightBin = {}, {}
totalDistance, totalWater, total_trials = {}, {}, {}
XtrackRunToLeftBin, XtrackRunToRightBin = {}, {}
instantSpeedLeftBin, instantSpeedRightBin = {}, {}
trackTimeToLeftBin, trackTimeToRightBin = {}, {}
sequence = {}

### COLORS we use RGB [0-1] not [0-255]. See www.colorhexa.com for conversion 
# old_old_palette = ['darkred', 'indianred', 'orangered', 'darkgreen', 'forestgreen', 'limegreen']
# old_palette = [(0.55, 0.0, 0.0),  (0.8, 0.36, 0.36),   (1.0, 0.27, 0.0), 
#                 (0.0, 0.39, 0.0),    (0.13, 0.55, 0.13),   (0.2, 0.8, 0.2)]


palette = {'RatF00': (0.4, 0.0, 0.0), 'RatF01': (0.55, 0.13, 0.13), 'RatF02': (0.8, 0.2, 0.2), 'RatF03': (1, 0.6, 0.6),
            'RatM00': (0.0, 0.4, 0.0), 'RatM01': (0.13, 0.55, 0.13), 'RatM02': (0.2, 0.8, 0.2), 'RatM03': (0.6, 1.0, 0.6)}

# needs to be properly implemented, setting is in behav_params for each session.
brainstatus = {'RatF00': 'normal', 'RatF01': 'lesion', 'RatF02': 'normal', 'RatF03': 'lesion',
               'RatM00': 'normal', 'RatM01': 'normal', 'RatM02': 'normal', 'RatM03': 'normal'}

animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
oldAnimalList = ['RatF00', 'RatF01', 'RatF02', 'RatM00', 'RatM01', 'RatM02']
lesionAnimalList = ['RatF01', 'RatF02', 'RatF03', 'RatM03']
for index, animal in enumerate(animalList):
    print(animal)
    if fnmatch.fnmatch(animal, 'RatF*'): rat_markers[animal]=[palette[animal], "$\u2640$", brainstatus_plot(brainstatus[animal])]
    elif fnmatch.fnmatch(animal, 'RatM*'): rat_markers[animal]=[palette[animal], "$\u2642$", brainstatus_plot(brainstatus[animal])]
    elif fnmatch.fnmatch(animal, 'Rat00*'): rat_markers[animal]=[(0.0, 0.0, 0.0), "$\u2426$", brainstatus_plot(brainstatus[animal])]
    else: print("error, this is not a rat you got here")

    for session in sorted(matchsession(animal, dist60+dist90+dist120 + TM20+TM10+TM2+TMrev2+TMrev10+TMrev20 +
                                                postVacs60+postVacs60BIS+postVacs90_1+postVacs90_2+postVacs120 + halfyear60+halfyear90_1+halfyear120+halfyear90_2 +
                                                lesionrev20+lesionrev10+lesionrev2+lesion2+lesion10+lesion20 + lesion60+lesion90+lesion120)):
        params[animal, session]     = get_from_pickle(root, animal, session, name="params.p")
        smoothMask[animal, session] = get_from_pickle(root, animal, session, name="mask.p")
        nb_runsBin[animal, session] = get_from_pickle(root, animal, session, name="nbRuns.p")
        speedRunToLeftBin[animal, session],  speedRunToRightBin[animal, session]  = get_from_pickle(root, animal, session, name="avgSpeed.p")
        maxSpeedLeftBin[animal, session],    maxSpeedRightBin[animal, session]    = get_from_pickle(root, animal, session, name="maxSpeed.p")
        timeStayInLeftBin[animal, session],  timeStayInRightBin[animal, session]  = get_from_pickle(root, animal, session, name="timeinZone.p")
        timeRunToLeftBin[animal, session],   timeRunToRightBin[animal, session]   = get_from_pickle(root, animal, session, name="timeRun.p")
        XtrackRunToLeftBin[animal, session], XtrackRunToRightBin[animal, session] = get_from_pickle(root, animal, session, name="trackPos.p")
        instantSpeedLeftBin[animal, session],instantSpeedRightBin[animal, session]= get_from_pickle(root, animal, session, name="trackSpeed.p")
        trackTimeToLeftBin[animal, session], trackTimeToRightBin[animal, session] = get_from_pickle(root, animal, session, name="trackTime.p")
        totalDistance[animal, session],      totalWater[animal, session], total_trials[animal, session] = get_from_pickle(root, animal, session, name="misc.p")
        sequence[animal, session] = get_from_pickle(root, animal, session, name="test.p")

    # for session in sorted(matchsession(animal, trainDist+TMtrain+postVacsTrain)): # halfyeartrain+lesionTrain
    #     params[animal, session] = get_from_pickle(root, animal, session, name="params.p")
    #     smoothMask[animal, session] = get_from_pickle(root, animal, session, name="mask.p")
    #     totalDistance[animal, session], totalWater[animal, session], total_trials[animal, session] = get_from_pickle(root, animal, session, name="misc.p")

# sessions pre lesion
data60, data90, data120 = separate_data(animalList, dist60+dist90+dist120, XtrackRunToLeftBin, XtrackRunToRightBin, 'Distance', params, 'avgrunspeed', True)
datat60, datat90, datat120 = separate_data(animalList, dist60+dist90+dist120, trackTimeToLeftBin, trackTimeToRightBin, 'Distance', params, 'avgrunspeed', True)
sdata60, sdata90, sdata120 = separate_data(animalList, dist60+dist90+dist120, instantSpeedLeftBin, instantSpeedRightBin, 'Distance', params, 'avgrunspeed', True)
sdatat60, sdatat90, sdatat120 = separate_data(animalList, dist60+dist90+dist120, trackTimeToLeftBin, trackTimeToRightBin, 'Distance', params, 'avgrunspeed', True)
datarev20, datarev10, datarev2, data2, data10, data20 = separate_data(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, XtrackRunToLeftBin, XtrackRunToRightBin, 'TM_ON', params, 'avgrunspeed', True)
datarevt20, datarevt10, datarevt2, datat2, datat10, datat20 = separate_data(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, trackTimeToLeftBin, trackTimeToRightBin, 'TM_ON', params, 'avgrunspeed', True)
sdatarev20, sdatarev10, sdatarev2, sdata2, sdata10, sdata20 = separate_data(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, instantSpeedLeftBin, instantSpeedRightBin, 'TM_ON', params, 'avgrunspeed', True)
sdatarevt20, sdatarevt10, sdatarevt2, sdatat2, sdatat10, sdatat20 = separate_data(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, trackTimeToLeftBin, trackTimeToRightBin, 'TM_ON', params, 'avgrunspeed', True)

# lesion sessions
datalesionrev20, datalesionrev10, datalesionrev2, datalesion2, datalesion10, datalesion20 = separate_data(animalList, lesionrev20+lesionrev10+lesionrev2+lesion2+lesion10+lesion20, XtrackRunToLeftBin, XtrackRunToRightBin, 'TM_ON', params, 'avgrunspeed', True)
datalesionrevt20, datalesionrevt10, datalesionrevt2, datalesiont2, datalesiont10, datalesiont20 = separate_data(animalList, lesionrev20+lesionrev10+lesionrev2+lesion2+lesion10+lesion20, trackTimeToLeftBin, trackTimeToRightBin, 'TM_ON', params, 'avgrunspeed', True)
sdatalesionrev20, sdatalesionrev10, sdatalesionrev2, sdatalesion2, sdatalesion10, sdatalesion20 = separate_data(animalList, lesionrev20+lesionrev10+lesionrev2+lesion2+lesion10+lesion20, instantSpeedLeftBin, instantSpeedRightBin, 'TM_ON', params, 'avgrunspeed', True)
sdatalesionrevt20, sdatalesionrevt10, sdatalesionrevt2, sdatalesiont2, sdatalesiont10, sdatalesiont20 = separate_data(animalList, lesionrev20+lesionrev10+lesionrev2+lesion2+lesion10+lesion20, trackTimeToLeftBin, trackTimeToRightBin, 'TM_ON', params, 'avgrunspeed', True)
datalesion60, datalesion90, datalesion120 = separate_data(animalList, lesion120+lesion60+lesion90, XtrackRunToLeftBin, XtrackRunToRightBin, 'Distance', params, 'avgrunspeed', True)
datalesiont60, datalesiont90, datalesiont120 = separate_data(animalList, lesion120+lesion60+lesion90, trackTimeToLeftBin, trackTimeToRightBin, 'Distance', params, 'avgrunspeed', True)
sdatalesion60, sdatalesion90, sdatalesion120 = separate_data(animalList, lesion120+lesion60+lesion90, instantSpeedLeftBin, instantSpeedRightBin, 'Distance', params, 'avgrunspeed', True)
sdatalesiont60, sdatalesiont90, sdatalesiont120 = separate_data(animalList, lesion120+lesion60+lesion90, trackTimeToLeftBin, trackTimeToRightBin, 'Distance', params, 'avgrunspeed', True)

RatF00


NameError: name 'brainstatus_plot' is not defined